# "Model Based Reinforcement Learning (MBRL)"
> "This is a summary of MBRL from ICML-2020 tutorial."

- toc: True
- branch: master
- badges: true
- comments: true
- categories: [fastpages, jupyter]
- image: images/some_folder/your_image.png
- hide: false
- search_exclude: true


This post is a summary of the model-based RL tutorial at ICML-2020. You can find the videos [here](https://sites.google.com/view/mbrl-tutorial). The pictures are from the slides in the talk.


## Introduction and Motivation

Having access to a model of the world and using it for decision making is a powerful idea. 
There are a lot of applications of MBRL in different areas like robotics (manipulation- what will happen by doing an action), 
self-driving cars (having a model of other agents decisions and future motions and act accordingly),
games (AlphaGo- search over different possibilities), Science ( chemical usecases),
and peration research and energy applications (how to allocate renewable energy in different points in time to meet the demand).

## Problem Statement

In sequential decision making, the agent will interact with the world by doing action $a$ and getting the next state $s$ and reward $r$.


<img src="images/rl.png" width="400">


We can write this problem as a Markov Decision Process (MDP) as follows:

- States $S \epsilon R^{d_S}$
- Actions $A \epsilon R^{d_A}$
- Reward function $R: S \times A \rightarrow R$
- Transition function $T: S \times A \rightarrow S$
- Discount $\gamma \epsilon (0,1)$
- Policy $\pi: S \rightarrow A$

The goal is to find a policy which maximizes the sum of discounted future rewards:
$$
\text{argmax}_{\pi} \sum_{t=0}^\infty \gamma^t R(s_t, a_t)
$$
subject to
$$
a_t = \pi(s_t) , s_{t+1}=T(s_t, a_t)
$$

How to solve this optimization problem?! 

- Collect data $D= \{ s_t, a_t, r_{t+1}, s_{t+1} \}_{t=0}^T$.
- Model-free: learn policy directly from data

$ D \rightarrow \pi$ e.g. Q-learning, policy gradient

- Model-based: learn model, then use it to **learn** or **improve** a policy 

$ D \rightarrow f \rightarrow \pi$ 


## What is a model?

a model is a representation that explicitly encodes knowledge about the structure of the environment and task.

This model can take a lot of different forms:

- A transition/dynamic model: $s_{t+1} = f_s(s_t, a_t)$
- A model of rewards: $r_{t+1} = f_r(s_t, a_t)$
- An inverse transition/dynamics model (which tells you what is the action to take and go from one state to the next state): $a_t = f_s^{-1}(s_t, s_{t+1})$
- A model of distance of two states: $d_{ij} = f_d(s_i, s_j)$
- A model of future returns: $G_t = Q(s_t, a_t)$ or $G_t = V(s_t)$

Typically when someone says MBRL, he/she means the firs two items.

<img src="images/model.png">

Sometimes we know the ground truth dynamics and rewards. Might as well use them! Like game environments or simulators like Mujoco, Carla, and so on.

But we don't have access to the model in all cases, so we need to learn the model. In cases like in robots, complex physical dynamics, and interaction with humans.




## How to use model?

In model-free RL agent we have a policy and learning algorithm like the figure below:

<img src="images/rl2.png">

In model-based RL we can use the model in three different ways:
- simulating the environment: replacing the environment with model and use it to generate data and use it to update the policy.
- Assisting the learning algorithm: modify the learning algorithm to use the model to interpret the data it is getting in a different way. 
- Strengthening the policy: allow the agent at test time to use the model to try out different actions before it commits to one of them (taking the action in the real world).

<img src="images/mbrl.png">

In general, to compare model-free and model-based:
    
<img src="images/mbrl_vs_mfrl.png">

## How to learn a model?


There are two different dimensions that are useful to pay attention to:
- representation of the features for the states that the model is being learned over them

- representation of the transition between states

<img src="images/learn_model.png">


In continue we take a look at different transition models.

### state-transition models

In some cases, we know equations of motion and dynamics but we don't know the exact parameters like mass. We can use system identification to estimate unknown parameters like mass. But these sort of cases require having a lot of domain knowledge about how exactly the system works.

<img src="images/learn_model2.png">
<img src="images/learn_model3.png">



In some cases that we don't know the dynamics of motion, we can simply use an MLP to get a concatenation of $s_t, a_t$ and output the next state $s_{t+1}$.

<img src="images/learn_model4.png">


In cases that we have some, not perfect, domain knowledge about the environment, we can use graph neural networks (GNNs) to model the agent (robot). For example in Mujoco we can model a robot (agent) with nodes as its body parts and edges as joint and learn the physics engine.

<img src="images/learn_model5.png">



### observation-transition models

In this cases, we don't have access to states (low level states like joint angles), but we have access to images. The MDP for this cases would be like this:

<img src="images/learn_model6.png">

So what can we do with this? 

- Directly predict transitions between observations (observation-transition models)

<img src="images/learn_model7.png">

- Reconstruct observation at every timestep: Using sth like LSTMs. Here we need to reconstruct the whole observation in each timestep. The images can be blurry in these cases.

<img src="images/learn_model8.png">

<img src="images/learn_model88.png">



### latent state-transition models

Another option when we have just access to observation is to instead of making transition between observations we can infere a latent state and then make transitions in that latent space (latent state-transition models) not in the observation space. It would be much faster than reconstructing the observation on every timestep. We take our initial observation or perhaps the last couple of observations and embed them into the latent state and then unroll it in time and do predictions in $z$ instead of $o$.

<img src="images/learn_model9.png">

Usually we use the observation and reconstruct it during training but at test time we can unroll it very quickly. we can also reconstruct observation at each timestep we want (not necessarily in all timesteps).

<img src="images/learn_model10.png">


### Structured latent state-transition models

Another thing that you can do if you have a little bit more domain knowledge is to add a little bit of structure into your latent state. For example, if you know that the scene that you are trying to model consists of objects, then you can try to actually explicitly detect those objects, segment them out and then learn those transitions between objects.

<img src="images/learn_model11.png">


### Recurrent value models

The idea is that when you unroll your latent-state, you additionally predict the value of the state at each point of the furture, in addition to reward. we can train the model without necessarily needing to train using observations, but just training it by predicting the value progressing toward actual observed values when you roll it out in the real environment.

<img src="images/learn_model12.png">

why this is useful? because some types of planners actually only need you to predict values rather than predicting states lime MCTS (monte carlo tree search).


### Non-Parametric models

So far we talked about parametric ways of learning the model. We can also use non-parametric methods like graphs.

For example the replay buffer that we use in off-policy methods can be seen as an approximation to a type of model where if you have enough data in your replay buffer then you sample from buffer, it basically access the density model over your transitions. You can use extra replay to basically get the same level performances you would get using a model based method which learns a parametric model.

<img src="images/learn_model13.png">

The other work we can do using data in buffer is to use data points and learn the transition between them and interpolate to find states between those states in buffer. Somehow learning a distribution and use it to generate new datapoints.

<img src="images/learn_model14.png">

Another form of non-parametric transition is a symbolic description which is popular in the planning community not in the deep learning community. 

<img src="images/learn_model15.png">

The other form of non-parametric models is gaussian processes which gives us strong predictions using very very small amout of data. PILCO is one example of these algorithms.

<img src="images/learn_model16.png">



## Model-based control and how to use a model?

We will be using this landscape of various methods and categories that exist, including some representative algorithms:

<img src="images/mbc1.png">

As we saw earlier, we can use the model in three different ways. In continue, we will see some examples of each case.

<img src="images/mbc.png" width="300">




### Simulating the environment

<img src="images/mbc2.png" width="150">

One way is to mix the real data with model-generated experience and then apply traditional model-free algorithms like Q-learning, policy gradient, etc. In this cases, the model offfers a larger and augmented training dataset.

**Dyna-Q** is an example that uses Q-learning with a learned model. Dyna does the traditional Q-learning updates on real transitions, and also use a model to create fictitious imaginary transitions from the real states and perform exactly the same Q-learning updates on those. So it's basically just a way to augment the experience.

<img src="images/mbc3.png">

This can also be applied in policy learning. We don't need to perform just a single step but multiple steps according to the **model** to generate experience even further away from the real data and do policy parameter updates entirely on this fictitious experiences.

<img src="images/mbc4.png">


### Assisting the learning algorithm

<img src="images/mbc5.png" width="150">

One important way that this can be done is to allow end-to-end training through our models. End-to-end training recently has been very successfull in improving and simplifying supervised learning methods in computer vision, NLP, etc. 

The question is "can we apply the same type of end-to-end approaches to RL?"

One example is just the policy gradient algorithm. Let's say we want to maximize the sum of discounted future reward of some parametric policy. We can write the objective function with respect to the policy parameters $\theta$

$$
 J(\theta) = \sum_{t=0}^{\infty} \gamma^t R(s_t, a_t)  , \quad a_t = \pi_{\theta}(s_t) , \quad s_{t+1} = T(s_t, a_t)
$$

Now we need to apply gradient ascent (for maximization) on policy gradient with respect to policy parameters $\theta  \rightarrow  \nabla_{\theta}J$.

So how can we calculate this $\nabla_{\theta}J$ ?

sampling-based methods have been proposed like REINFORCE to estimate this gradient. But the problem with them is that they cannot have a very high variance and they often require the policy to have some randomness to it to make decisions. This can be unfavorable.

<img src="images/mbc6.png">



Accurate and smooth models, aside from imaginary experiences, offer derivatives:

$$
s_{t+1} = f_s(s_t, a_t) \quad  r_t = f_r(s_t, a_t)
$$

$$
\nabla_{s_t}(s_{t+1}), \quad \nabla_{a_t}(s_{t+1}), \quad \nabla_{s_t}(r_t), \quad \nabla_{a_t}(r_t), \quad ...
$$

And they are able to answer questions such as: *how do small changes in action change next state or reward any of other quantities?*

<img src="images/mbc7.png" width="150">


Why is this useful? This is useful because it will allow us to do this type end-to-end differentiation algorithms like **back-propagation**.

Let's rewrite our objective function using models:

$$
 J(\theta) \approx \sum_{t=0}^{H} \gamma^t r_t  , \quad a_t = \pi_{\theta}(s_t) , \quad s_{t+1} = f_s(s_t, a_t), \quad r_t=f_r(s_t,a_t)
$$


So how we can use this derivatives to calculate $\nabla_{\theta}J$ ?




<img src="images/mbc8.png">

The highlighted derivatives are easy to calculate using some libraries like PyTprch or TensorFlow.

By calculating $\nabla_{\theta}J$ in this way:

**pros**:
+ The policy gradient that we get is actually a deterministic quantity and there is no variance to it. 
+ It can support potentially much longer term credit assignment

**cons**:
- It is prone to local minima
- Poor conditioning (vanishing/exploding gradients)

Here are two examples to use model-based back-propagation (derivatives) either along real or model-generated ttrajectories to do end to end training:

<img src="images/mbc9.png">

- real trajectories are safer, but need to be from the current policy parameters (so it’s less sample-efficient)
- model-generated trajectories allow larger policy changes without interacting with the real world, but might suffer more from model inaccuracies



### Strengthening the policy 

So far we talked about the first two ways of using a model in RL. These two ways are in category of **Background Planning**. 

There is another category based on the *Sutton and Barto (2018)- Reinforcement Learning: An Introduction* categorization, called **Decision-Time Planning**, which is a unique option we have available in model-based settings.

#### What is the difference between background and decision-time planning?
In background planning, we can think of it as answering the question "how do I learn how to act in any possible situation to succeed and reach the goal?"

<img src="images/mbc10.png" width="300">

- The optimization variables are parameters of a policy or value function or ... and are trained using expectation over all possible situations.
- Conceptually we can think of background planning as learning a set of habits that I could just reuse.
- We can think of background planning as learning fast type of thinking.


In decision-time planning, we want to answer the question "what is the best sequence of actions just for my current situation to succeed or reach the goal?"

<img src="images/mbc11.png" width="300">

- The optimization parameters are just a sequence of actions or states.
- Conceptually we can think of decision-time planning as finding our consciously improvising just for the particular situation that we find ourself in.
- We can think of decision-time planning as learning slow type of thinking.


Why use one over the other?

<img src="images/mbc12.png">


- *Act on most recent state of the world*: decision-time planning is just concerned about the current state in finding the sequence of actions. You can act based on the most recent state of the world. By contrast, in background planning the habits may be stale and might take a while to get updated as the world's changes.

- *Act without any learning*: decision-time planning allows us to act without any learning at all. There is no need for policy or value networks before we can start making decisions. It is just an optimization problem as long as you have the model.

- *Competent in unfamiliar situations*: if you find yourself in situations that are far away from where you were training, your set of habits or policy network might not have competence (the ability to do something successfully or efficiently) there. So you don't have any information to act or are very uncertain or even in the worst case, it will with confidence make decisions that just potentionally make no sense. This is out of distribution and generalization problem. In these cases the decision-time planning would be more beneficial.

- *Independent of observation space*: another advantage of decision-time planning is that it is also independent of the observation space that you decide on. In background methods we need to consider some encoding or description of the state, joint angles or pixels or graphs, into our policy function. This decisions may play a large role on the total learning performance. When something is not working, you will not really know that is it because of the algorithm, or state space which doesn't contain enough information. In contrast, decision-time planning avoid this confounded, which in practice can be actually quite useful whn you're prototyping new methods.

<img src="images/mbc13.png" width="500">

- *Partial observability*: decision-time plannings have some issues with it. They assume that you know the full state of the world when you're making the plan. So it's hard to hide information from decision-time planners. It is possible but it is more costly.

- *Fast computation at deployment*: decision-time planners require more computation. It is not just evaluating a habbit, but it needs more thinking.

- *Predictability and coherence*: decision-time planners do some actions which are not necessarily predictable or coherent. Because you are consciously thinking about each foot step, you might not have a plan that's exactly the same. So you may have a very chaotic behavior that still succeeds. In contrast, the background planning, because it learns a set of habbits, it can perform a very regular behavior.

- *Same for discrete and contiinuous actions*: background planning has a very unified treatment of discrete and continuous actions which is conceptually simpler. In decision-time planning, there are different algorithms for discrete and continuous actions. we will see in the following sections more about them.

We can also mix and match background and decision-time plannings.

#### What is the difference between discrete and continuous planning?

It depends on the problem which you want to solve. So it is not a choice that you can make. For example in controlling a robot, the actions might be the torques for the motors (continuous), or in biomechanical settings it might be muscle excitations (continuous), or in medical problems the treatment that should be applied (discrete).

The distinction between discrete and continuous actions is not significant for background planning methods. 
- You just learn a stochastic policies that sample either from discrete or continuous distributions.

$$
a \sim \pi(.|s) \quad \leftarrow Gaussian, categorical, ...
$$

- Backpropagation is still possible via some reparametrization techniques. See *Jang et al (2016). Categorical reparametrization with Gumbel-Softmax* for an example.

In either of these cases (continuous and discrete in background planning methods), because you are optimizing over expectations, your final objective and optimization problem is still smooth wrt the policy parameters.

$$
J(\theta) = E_{\pi}[\sum_t r_t], \quad a_t \sim \pi(.|s_t, \theta)
$$

But for decision-time planning, this distinction leads to specialized methods for discrete and continuous actions: discrete search or continuous trajectory optimization.

Let's see some examples to be able to compare them.

<img src="images/mbc14.png" width="400">



##### MCTS (monte carlo tree search)

This algorithm is in discrete actions group and is used in alpha-go and alpha-zero. You keep track of Q-value, which is long term reward, for all states and actions ideally that you want to consider. And also the number of times that the state and action has been previously visited. 

1. Initialize $Q_0(s, a) = 0, N_0(s, a)=0, k=0$

<img src="images/mbc15.png" width="150">

2. Expansion: Starting from the current situation and expand nodes and selecting actions according to a search policy: 

$$\pi_k(s) = Q_k(s,a)$$

<img src="images/mbc16.png" width="150">

3. Evaluation: When a new node is reached, estimate its long-term value using Monte-Carlo rollouts

<img src="images/mbc17.png" width="200">

4. Backup: Propagate the Q-values to parent nodes:

$$
Q_{k+1}(s, a) = \frac{Q_k(s,a) N_k(s,a) + R}{N_k(s,a)+1}
$$

$$
N_{k+1}(s,a) = N_k(s,a)+1
$$

<img src="images/mbc18.png" width="300">


5. Repeat Steps 2-4 until search budget is exhausted.
$$
k = k + 1
$$

<img src="images/mbc19.png"  width="300">



##### Trajectory Optimization

Instead of keeping track of a tree of many possibilities, you just keep track of one possible action sequence.

1. Initialize $a_0, ..., a_H$ from guess

<img src="images/mbc20.png" width="200">

2. **Expansion**: execute sequence of actions $a = a_0, ..., a_H$ to get a sequence of states $s_1, ..., s_H$

<img src="images/mbc21.png" width="200">

3. **Evaluation**: get trajectory reward $J(a) = \sum_{t=0}^H r_t$

4. **Back-propagation**: because everything is differentiable, you can just calculate the gradient of the reward via back-propagation using reward model derivatives and transition model derivatives.

$$
\nabla_a J = \sum_{t=0}^H \nabla_a r_t
$$
$$
\nabla_a r_t = \nabla_s f_r(s_t, a_t) \nabla_a s_t + \nabla_a f_r (s_t, a_t)
$$
$$
\nabla_a s_t = \nabla_a f_s(s_{t-1}, a_{t-1}) + \nabla_s f_s(s_{t-1}, a_{t-1})\nabla_a s_{t-1}
$$
$$
\nabla_a s_{t-1} = ...
$$

5. Update all actions via gradient ascent $ a \leftarrow a + \nabla_a J$ and repeat steps 2-5.

<img src="images/mbc22.png" width="200">

The differences between discrete and continuous actions can be summarized as follows:

<img src="images/mbc23.png">



The continuous example we saw above can be categorize in **shooting methods**.

#### Variety and motivations of continuous planning methods

Why so many variations? They all try to mitigate the issues we looked at like:
- Sensitivity and poor conditioning
- Only reaches local optimum
- Slow convergence

Addressing each leads to a different class of methods. 

<img src="images/mbc24.png" width="200">



##### Sensitivity and poor conditioning

<img src="images/mbc24-2.png" width="200">

**Shooting methods** that we have seen have this particular issue that small changes in early actions lead to very large changes downstream.

<img src="images/mbc25.png" width="200">

By expanding the objective function, this can be understood more clearly.

$$
\max_{a_0,...,a_H} \sum_{t=0}^H r(s_t, a_t), \quad s_{t+1} = f(s_t, a_t)
$$
$$
\sum_{t=0}^H r(s_t, a_t) = r(s_0, a_0) + r(f(s_0, a_0), a_1)+...+r(f(f(...),...), a_H)
$$

It means that each state implicitly is dependent on all actions that came before it. This is similar to exploding/vanishing gradient ptoblem in RNNs that hurts long-term credit assignment. But unlike the RNN training, we cannot change the transition function because it is dictated to us by the environment. 

<img src="images/mbc26.png" width="400">



To address this problem, **Collocation** is introduced, which is optimizing for states and/or actions *directly*, instead of actions only. So we have different set of parameters that we are optmizing over.

$$
\max_{s_0,a_0,...,s_H,a_H} \sum_{t=0}^H r(s_t, a_t), \quad ||s_{t+1} - f(s_t, a_t) || = 0 \leftarrow \text{explicit optimization constraint}
$$

It is an explicit contrained optimization problem, rather than just beeng satisfied by construction as in shooting methods.

As a result, you only have pairwise dependencies between variables, unlike the dense activity graph in the previous figure for shooting methods.

<img src="images/mbc27.png" width="400">

An these methods have:
- Good conditioning: changing $s_0, a_0$ has similar effect as changing $s_H, a_H$.
- Larger but easier to optimize search space. It is useful for contact-rich problems such as some robotics applications.

##### Only reaches local optimum

<img src="images/mbc28.png" width="200">

Some approaches try to avoid local optima like sampling based methods: Cross-Entropy Methods (CEM) and $\text{PI}^2$.

For example in CEMs, instead of just maintaining the optimal trajectory, it maintains the mean and covariance of that optimal trajectory. 

<img src="images/mbc29.png" width="500">
<img src="images/mbc30.png" width="500">
<img src="images/mbc31.png" width="500">


Despite being very simple, this works surprisingly well and has very nice guarantees on performance.

Why does this work?
- Search space of decision-time plans much smaller than space of policy parameters: ex. 30x32 vs 32x644x32
- More feasible plans than policy parameters


##### Slow convergence

<img src="images/mbc32.png" width="200">

Gradient descent is too slow to converge and we need to wait thousands-millions of iterations to train a policy. But this is too long for a one-time plan that we want to through it away after.

Can we do something like Newton’s method for trajectory optimization like as non-linear optimization? YES!

We can approximate transitions with linear functions and rewards with quadratics: 

$$
\max_{a_0,...,a_H} \sum_{t=0}^H r_t, \quad s_{t+1} = f_s(s_t, a_t), \quad r_t=f_r(s_t, a_t)
$$

$$
f_s(s_t, a_t) \approx As_t + Ba_t, \quad f_r(s_t, a_t) \approx s_t^TQs_t + a_t^TRa_t
$$

Then it becomes Linear-Quadratic Regulator (LQR) problem and can be solved exactly.

For iLQR, locally approximate the model around current solution, solve LQR problem to update solution, and repeat.

For Differential dynamic programming (DDP) is similar, but with higher-order expansion of $f_s$.